# GRU Classification with CR Dataset
<hr>

We will build a text classification model using GRU model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/CR/CR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(3775, 3)


,sentence,label,split
0,weaknesses are minor the feel and layout of th...,0,train
1,many of our disney movies do n 't play on this...,0,train
2,player has a problem with dual layer dvd 's su...,0,train
3,i know the saying is you get what you pay for ...,0,train
4,will never purchase apex again .,0,train
...,...,...,...
3770,"so far , the anti spam feature seems to be ver...",1,train
3771,i downloaded a trial version of computer assoc...,1,train
3772,i did not have any of the installation problem...,1,train
3773,their products have been great and have saved ...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  3775 non-null   object
 1   label     3775 non-null   int32 
 2   split     3775 non-null   object
dtypes: int32(1), object(2)
memory usage: 73.9+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,1368,1368
1,2407,2407


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

"weaknesses are minor the feel and layout of the remote control are only so so . it does n 't show the complete file names of mp3s with really long names . you must cycle through every zoom setting ( 2x , 3x , 4x , 1 2x , etc . ) before getting back to normal size sorry if i 'm just ignorant of a way to get back to 1x quickly ."

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  will never purchase apex again .
Into a sequence of int: [72, 194, 285, 207, 286]
Into a padded sequence: [ 72 194 285 207 286   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
and 3
i 4
it 5
to 6
a 7
is 8
of 9
this 10
5336


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## GRU Model

In [11]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               140544    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 440,673
Trainable params: 440,673
Non-trainable params: 0
_________________________________________________________________


In [33]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [34]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=30, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/30
107/107 [==============================] - 43s 309ms/step - loss: 0.6296 - accuracy: 0.6404 - val_loss: 0.4986 - val_accuracy: 0.7513
Epoch 2/30
107/107 [==============================] - 24s 229ms/step - loss: 0.2904 - accuracy: 0.8794 - val_loss: 0.4936 - val_accuracy: 0.7672
Epoch 3/30
107/107 [==============================] - 22s 203ms/step - loss: 0.1148 - accuracy: 0.9613 - val_loss: 0.5883 - val_accuracy: 0.7487
Epoch 4/30
107/107 [==============================] - 21s 201ms/step - loss: 0.0466 - accuracy: 0.9847 - val_loss: 0.6880 - val_accuracy: 0.7619
Epoch 5/30
107/107 [==============================] - 21s 200ms/step - loss: 0.0245 - accuracy: 0.9941 - val_loss: 0.8613 - val_accuracy: 0.7381
Epoch 6/30
107/107 [==============================] - 22s 201ms/step - loss: 0.0092 - accuracy: 0.9970 - val_loss: 1.0451 - val_accuracy: 0.7593
Epoch 7/30
107/107 [==============================] - 21s 199ms/step - loss: 0.0040 - accuracy: 0.9986 - val_loss: 1.

Epoch 7/30
107/107 [==============================] - 36s 333ms/step - loss: 0.0135 - accuracy: 0.9962 - val_loss: 0.9987 - val_accuracy: 0.7804
Epoch 8/30
107/107 [==============================] - 36s 332ms/step - loss: 0.0041 - accuracy: 0.9990 - val_loss: 1.1415 - val_accuracy: 0.7804
Epoch 9/30
107/107 [==============================] - 35s 330ms/step - loss: 0.0027 - accuracy: 0.9986 - val_loss: 1.2575 - val_accuracy: 0.7857
Epoch 10/30
107/107 [==============================] - 35s 330ms/step - loss: 0.0020 - accuracy: 0.9984 - val_loss: 1.2993 - val_accuracy: 0.7751
Epoch 11/30
107/107 [==============================] - 35s 328ms/step - loss: 0.0020 - accuracy: 0.9983 - val_loss: 1.3994 - val_accuracy: 0.7778
Epoch 12/30
107/107 [==============================] - 35s 330ms/step - loss: 0.0014 - accuracy: 0.9991 - val_loss: 1.4575 - val_accuracy: 0.7831
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 81.21693134307861
Training 6

Epoch 12/30
107/107 [==============================] - 86s 800ms/step - loss: 7.1514e-04 - accuracy: 0.9997 - val_loss: 1.3317 - val_accuracy: 0.7958
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 82.49337077140808
Training 10: 
Epoch 1/30
107/107 [==============================] - 105s 883ms/step - loss: 0.6272 - accuracy: 0.6474 - val_loss: 0.4421 - val_accuracy: 0.7878
Epoch 2/30
107/107 [==============================] - 72s 673ms/step - loss: 0.2698 - accuracy: 0.8879 - val_loss: 0.4446 - val_accuracy: 0.8143
Epoch 3/30
107/107 [==============================] - 63s 589ms/step - loss: 0.1214 - accuracy: 0.9597 - val_loss: 0.5284 - val_accuracy: 0.7825
Epoch 4/30
107/107 [==============================] - 62s 578ms/step - loss: 0.0701 - accuracy: 0.9792 - val_loss: 0.7685 - val_accuracy: 0.7905
Epoch 5/30
107/107 [==============================] - 61s 571ms/step - loss: 0.0308 - accuracy: 0.9889 - val_loss: 0.9923 - val_accuracy: 

## Summary

In [35]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,76.719576,79.894179,78.571427,82.804233,81.216931,81.697613,77.453583,79.045093,82.493371,81.43236,80.132837


In [36]:
report = record
report = report.to_excel('GRU_CR.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [37]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [38]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [39]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [40]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 5046 words present from 5336 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [41]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [42]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.7279541 , -1.3389816 , -0.41429527, ...,  0.25257593,
        -0.89658364, -0.78454617],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## GRU Model

In [43]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [44]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_34 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_34 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 140,673
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [45]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [46]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/40
107/107 [==============================] - 49s 356ms/step - loss: 0.6286 - accuracy: 0.6436 - val_loss: 0.5425 - val_accuracy: 0.7302
Epoch 2/40
107/107 [==============================] - 33s 313ms/step - loss: 0.4783 - accuracy: 0.7621 - val_loss: 0.4727 - val_accuracy: 0.7672
Epoch 3/40
107/107 [==============================] - 32s 299ms/step - loss: 0.3967 - accuracy: 0.8207 - val_loss: 0.5047 - val_accuracy: 0.7672
Epoch 4/40
107/107 [==============================] - 31s 293ms/step - loss: 0.3805 - accuracy: 0.8338 - val_loss: 0.4941 - val_accuracy: 0.7725
Epoch 5/40
107/107 [==============================] - 32s 296ms/step - loss: 0.3284 - accuracy: 0.8569 - val_loss: 0.4671 - val_accuracy: 0.7884
Epoch 6/40
107/107 [==============================] - 33s 309ms/step - loss: 0.2972 - accuracy: 0.8701 - val_loss: 0.4906 - val_accuracy: 0.7751
Epoch 7/40
107/107 [==============================] - 33s 310ms/step - loss: 0.2807 - accuracy: 0.8816 - val_loss: 0.

107/107 [==============================] - 27s 249ms/step - loss: 0.2603 - accuracy: 0.8893 - val_loss: 0.6280 - val_accuracy: 0.7698
Epoch 9/40
107/107 [==============================] - 27s 248ms/step - loss: 0.2284 - accuracy: 0.8965 - val_loss: 0.5804 - val_accuracy: 0.7778
Epoch 10/40
107/107 [==============================] - 26s 248ms/step - loss: 0.1856 - accuracy: 0.9250 - val_loss: 0.6722 - val_accuracy: 0.7593
Epoch 11/40
107/107 [==============================] - 26s 246ms/step - loss: 0.2162 - accuracy: 0.9120 - val_loss: 0.6150 - val_accuracy: 0.7751
Epoch 12/40
107/107 [==============================] - 26s 247ms/step - loss: 0.1686 - accuracy: 0.9327 - val_loss: 0.7688 - val_accuracy: 0.7698
Epoch 13/40
107/107 [==============================] - 26s 246ms/step - loss: 0.1110 - accuracy: 0.9620 - val_loss: 0.7036 - val_accuracy: 0.7566
Epoch 14/40
107/107 [==============================] - 26s 244ms/step - loss: 0.1280 - accuracy: 0.9504 - val_loss: 0.7041 - val_accuracy

Epoch 10/40
107/107 [==============================] - 39s 361ms/step - loss: 0.1869 - accuracy: 0.9228 - val_loss: 0.5842 - val_accuracy: 0.7613
Epoch 11/40
107/107 [==============================] - 38s 352ms/step - loss: 0.1735 - accuracy: 0.9238 - val_loss: 0.7467 - val_accuracy: 0.7639
Epoch 12/40
107/107 [==============================] - 37s 350ms/step - loss: 0.1423 - accuracy: 0.9458 - val_loss: 0.7044 - val_accuracy: 0.7454
Epoch 13/40
107/107 [==============================] - 37s 349ms/step - loss: 0.1687 - accuracy: 0.9292 - val_loss: 0.7855 - val_accuracy: 0.7639
Epoch 14/40
107/107 [==============================] - 37s 348ms/step - loss: 0.1025 - accuracy: 0.9707 - val_loss: 0.8180 - val_accuracy: 0.7639
Epoch 15/40
107/107 [==============================] - 38s 351ms/step - loss: 0.0959 - accuracy: 0.9638 - val_loss: 0.7940 - val_accuracy: 0.7586
Epoch 16/40
107/107 [==============================] - 38s 357ms/step - loss: 0.0774 - accuracy: 0.9757 - val_loss: 1.0218 -

107/107 [==============================] - 42s 390ms/step - loss: 0.1662 - accuracy: 0.9344 - val_loss: 0.7857 - val_accuracy: 0.7401
Epoch 12/40
107/107 [==============================] - 41s 387ms/step - loss: 0.1475 - accuracy: 0.9466 - val_loss: 0.9047 - val_accuracy: 0.7347
Epoch 13/40
107/107 [==============================] - 41s 387ms/step - loss: 0.1025 - accuracy: 0.9643 - val_loss: 1.0146 - val_accuracy: 0.7268
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 78.24933528900146

        acc1       acc2       acc3       acc4       acc5       acc6  \
0  78.835976  79.100531  80.423278  79.365081  84.656084  78.249335   

        acc7       acc8       acc9      acc10        AVG  
0  79.575598  80.636603  84.880638  78.249335  80.397246  



## Summary

In [47]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,78.835976,79.100531,80.423278,79.365081,84.656084,78.249335,79.575598,80.636603,84.880638,78.249335,80.397246


In [48]:
report = record2
report = report.to_excel('GRU_CR_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## GRU Model

In [49]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [50]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_45 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_45 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_45 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 440,673
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [51]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [52]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/40
107/107 [==============================] - 80s 642ms/step - loss: 0.6326 - accuracy: 0.6375 - val_loss: 0.4725 - val_accuracy: 0.7725
Epoch 2/40
107/107 [==============================] - 62s 575ms/step - loss: 0.3715 - accuracy: 0.8356 - val_loss: 0.4138 - val_accuracy: 0.7989
Epoch 3/40
107/107 [==============================] - 62s 584ms/step - loss: 0.1893 - accuracy: 0.9380 - val_loss: 0.5095 - val_accuracy: 0.8069
Epoch 4/40
107/107 [==============================] - 62s 581ms/step - loss: 0.0864 - accuracy: 0.9733 - val_loss: 0.6016 - val_accuracy: 0.7698
Epoch 5/40
107/107 [==============================] - 62s 583ms/step - loss: 0.0642 - accuracy: 0.9823 - val_loss: 0.7925 - val_accuracy: 0.7884
Epoch 6/40
107/107 [==============================] - 62s 582ms/step - loss: 0.0376 - accuracy: 0.9898 - val_loss: 0.9291 - val_accuracy: 0.7910
Epoch 7/40
107/107 [==============================] - 62s 583ms/step - loss: 0.0172 - accuracy: 0.9934 - val_loss: 0.

107/107 [==============================] - 82s 767ms/step - loss: 0.0012 - accuracy: 0.9992 - val_loss: 1.0675 - val_accuracy: 0.8280
Epoch 16/40
107/107 [==============================] - 83s 775ms/step - loss: 0.0015 - accuracy: 0.9993 - val_loss: 1.1205 - val_accuracy: 0.8307
Epoch 17/40
107/107 [==============================] - 82s 765ms/step - loss: 0.0015 - accuracy: 0.9991 - val_loss: 1.1570 - val_accuracy: 0.8307
Epoch 18/40
107/107 [==============================] - 82s 763ms/step - loss: 0.0022 - accuracy: 0.9985 - val_loss: 1.1898 - val_accuracy: 0.8307
Epoch 19/40
107/107 [==============================] - 81s 761ms/step - loss: 0.0014 - accuracy: 0.9989 - val_loss: 1.2075 - val_accuracy: 0.8333
Epoch 20/40
107/107 [==============================] - 81s 760ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 1.2189 - val_accuracy: 0.8333
Epoch 21/40
107/107 [==============================] - 82s 765ms/step - loss: 0.0017 - accuracy: 0.9990 - val_loss: 1.2346 - val_accurac

Epoch 11/40
107/107 [==============================] - 73s 686ms/step - loss: 0.0051 - accuracy: 0.9987 - val_loss: 1.7103 - val_accuracy: 0.7427
Epoch 12/40
107/107 [==============================] - 73s 687ms/step - loss: 0.0073 - accuracy: 0.9982 - val_loss: 1.7315 - val_accuracy: 0.7401
Epoch 13/40
107/107 [==============================] - 73s 687ms/step - loss: 0.0020 - accuracy: 0.9996 - val_loss: 1.9313 - val_accuracy: 0.7427
Epoch 14/40
107/107 [==============================] - 73s 681ms/step - loss: 0.0011 - accuracy: 0.9992 - val_loss: 2.0014 - val_accuracy: 0.7454
Epoch 15/40
107/107 [==============================] - 74s 690ms/step - loss: 0.0017 - accuracy: 0.9987 - val_loss: 2.0810 - val_accuracy: 0.7401
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 74.53581094741821
Training 9: 
Epoch 1/40
107/107 [==============================] - 95s 781ms/step - loss: 0.6237 - accuracy: 0.6493 - val_loss: 0.4844 - val_accuracy: 0.

## Summary

In [53]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,80.687833,82.010579,86.243385,84.126985,80.423278,82.228118,79.310346,74.535811,80.901855,79.575598,81.004379


In [54]:
report = record3
report = report.to_excel('GRU_CR_3.xlsx', sheet_name='dynamic')